# SITCOM-1062 - M1M3 Inertia Compensation System

## Data Summary

[20230728] :
- BLOCK-82, a set of different slews in azimuth at fixed elevation followed but a set of different slews in elevation at fixed azimuth at 30% maximum motion settings.  
- BLOCK-5, use the Scheduler for observation simulations using M1M3 in closed look. Assuming 30% motions settings.

[20230802] : 
- We ran BLOCK-82 with different maximum motion settings.
    - 20% max in both axes
    - 30% max in both axes
    - 30% max in elevation and 40% max in azimuth
    - 30% max in elevation and 50% max in azimuth
    
(!) Why did we go only until 30% max in elevation again? 

[20230728]: https://confluence.lsstcorp.org/display/LSSTCOM/23.07.28+-+M1M3+Test+Log
[20230802]: https://confluence.lsstcorp.org/display/LSSTCOM/23.08.02+-+M1M3+Test+Log

## Prepare Notebook

For this notebook you will need to have `summit_utils` installed and running with the proper version.  
The current version for `summit_utils` is `tickets/DM-41232` until that ticket is merged/done.  
Otherwise, use `sitcom-performance-analysis` or `develop` branches. 

In [ ]:
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
from pathlib import Path

from lsst.ts.xml.tables.m1m3 import HP_COUNT

from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.summit.utils.m1m3 import inertia_compensation_system as m1m3_ics
from lsst.sitcom.vandv.m1m3.sitcomtn092 import (
    correlation_map,
    merge_csvs,
    multiaxis_plots,
    singleaxis_plots,
)

In [ ]:
day_obs_list = [
    20230728,
    20230802,
]

file_pattern = "m1m3_ics_{day_obs}.csv"
output_file = Path("m1m3_ics.csv")
data_folder = Path("./data")
plot_folder = Path("./plots")

## Generate tables with historical data

In [ ]:
event_maker = TMAEventMaker()
data_folder.mkdir(parents=True, exist_ok=True)
plot_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# The amount of output on this cell might break the plots.
# Restart the kernel and run the notebook again to get them displayed.
for day_obs in day_obs_list:
    file_path = Path(data_folder) / file_pattern.format(day_obs=day_obs)
    if file_path.exists():
        print(f"File exists: {file_path}\n Skipping data processing.")
        continue
    else:
        temp_df = m1m3_ics.evaluate_m1m3_ics_day_obs(day_obs, event_maker)
        temp_df.to_csv(file_path)
        del temp_df

## Merge datasets into a single one

In [ ]:
df = merge_csvs(data_folder, file_pattern, day_obs_list)
df.to_csv(data_folder / output_file)

## What data can be correlated with the HP forces?

For this, I will start with a correlation map.  
This migh give me an idea of what is related to what.  
I will start by replacing the boolean values of `ics_enabled` with numerical ones to allow gathering correlation.  
Then, I will temporarily drop the columns that I know don't have any correlation with the ICS performance.  
Finally, I will display the correlation map that will give us some impressions on the possible correlations.   

In [ ]:
df["ics_enabled"] = df["ics_enabled"].apply(lambda x: 1 if x else -1)

In [ ]:
subset_of_columns = [
    "time_duration",
    "az_start",
    "az_end",
    "az_extreme_vel",
    "az_extreme_torque",
    "az_diff",
    "el_start",
    "el_end",
    "el_extreme_vel",
    "el_extreme_torque",
    "el_diff",
    "ics_enabled",
]

subset_of_columns.extend([f"measuredForceMin{hp}" for hp in range(HP_COUNT)])
subset_of_columns.extend([f"measuredForceMax{hp}" for hp in range(HP_COUNT)])
subset_of_columns.extend([f"measuredForceMean{hp}" for hp in range(HP_COUNT)])
subset_of_columns.extend([f"measuredForceStd{hp}" for hp in range(HP_COUNT)])

In [ ]:
correlation_map(df, subset_of_columns)

From the plot above, here are a few curious points:

* ``measuredForceMin3`` and ``measuredForceMin4`` have anti-correlation while all the other ``measuredForceMin`` have direct correlation.  

* ``measuredForceMax`` forces are anti-correlated with almost all ``measuredForceMin``, as expected. However, ``measuredForceMin1`` and ``measuredForceMin4`` seem to be flippled.  
    
* ``el_extreme_vel`` seems to have stronger impact on measured forces than ``az_extreme_vel``  

## Expected performance during constant speed

Ideally, the Inertia Compensation System should offload all the forces from the Hard-Points to the Force Balance System when accelerating or when moving at constant velocity. Let's have a look at the first case first. Let me create a plot of the `measuredForceMean` values versus `az_extreme_torque` and `el_extreme_torque`.

In [ ]:
df["abs_az_extreme_vel"] = df["az_extreme_vel"].abs()
df["abs_az_extreme_torque"] = df["az_extreme_torque"].abs()
df["abs_el_extreme_vel"] = df["el_extreme_vel"].abs()
df["abs_el_extreme_torque"] = df["el_extreme_torque"].abs()

During the study, we identified an outlier that had `abs_el_extreme_vel` and `abs_el_extreme_torque` that had values inconsistently high. Since it was a single point, I will ignore it for now:

In [ ]:
print(f"Data-frame size before filtering: {df.index.size}")
df = df[df["abs_el_extreme_vel"] < 10]
print(f"Data-frame size after filtering: {df.index.size}")

In [ ]:
el_only_df = df[df["abs_az_extreme_vel"] < 0.02]
az_only_df = df[df["abs_el_extreme_vel"] < 0.02]

print(f"Total number of slews: {df.index.size}")
print(f"Number of elevation-only slews: {el_only_df.index.size}")
print(f"Number of azimuth-only slews: {az_only_df.index.size}")
print(f"Sum of the two above: {el_only_df.index.size + az_only_df.index.size}")

## El Only Motion Analysis

In [ ]:
multiaxis_plots(el_only_df, "abs_el_extreme_vel", "measuredForceMean")

In [ ]:
multiaxis_plots(el_only_df, "abs_el_extreme_torque", "measuredForceMax")

In [ ]:
only_ics_enabled = el_only_df[el_only_df["ics_enabled"] > 0]
singleaxis_plots(only_ics_enabled, "abs_el_extreme_torque", "measuredForceMax")

## Az Only Motion Analysis

In [ ]:
multiaxis_plots(az_only_df, "abs_az_extreme_vel", "measuredForceMean")

In [ ]:
multiaxis_plots(az_only_df, "abs_az_extreme_torque", "measuredForceMax")

In [ ]:
only_ics_enabled = az_only_df[az_only_df["ics_enabled"] > 0]
singleaxis_plots(only_ics_enabled, "abs_az_extreme_torque", "measuredForceMax")